# Welcome to the adversarial chatbot conversation Notebook!

Here we connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up the keys

We have created API keys for Anthropic and Google in addition to OpenAI.

We set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```


In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAZ


In [ ]:
# Connect to OpenAI, Anthropic, and Google APIs
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## An adversarial conversation between Chatbots..

Basic prompts structure:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

For a longer conversation history we have:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [6]:
# Let's make a conversation between GPT-4o-mini, Claude-3-haiku and Gemini-2.0-flash
# We'll use GPT-4o-mini as the argumentative chatbot, Claude-3-haiku as the polite chatbot,
# and Gemini-2.0-flash as a neutral chatbot.
# The conversation will start with a simple greeting and then proceed with the chatbots responding to each other.
# # We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are a neutral chatbot. You try to keep the conversation going \
without taking sides. You respond to the other person in a neutral way, \
but you also try to keep the conversation interesting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hello"]

In [8]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [9]:
call_gpt()

"I mean, hello again. It’s not like we totally didn’t just say that. Let's just get to the point, shall we?"

In [10]:
def call_claude():
    messages = []
    for gpt, claude_message, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": gemini_message})
    messages.append({"role": "user", "content": gpt_messages[-1]}) # Last message from GPT
    messages.append({"role": "user", "content": gemini_messages[-1]}) # Last message from Gemini
    print(messages)
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [11]:
call_claude()

[{'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hi'}, {'role': 'user', 'content': 'Hello'}, {'role': 'user', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hello'}]


"Hello! It's nice to meet you. How are you doing today?"

In [32]:
""" def call_gemini():
    history = []
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        history.append({"role": "user",  "parts": [gpt]})      
        history.append({"role": "user",  "parts": [claude]})   
        history.append({"role": "model", "parts": [gemini]})    

    model = google.generativeai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=gemini_system
    )
    # Start a chat with prior history, then send the next user prompt(s)
    chat = model.start_chat(history=history)

    #  Last two inputs (treat them as user turns so Gemini responds next)
    next_user_input = (
        gpt_messages[-1] + "\n" + claude_messages[-1]
    )
    response = chat.send_message(next_user_input)

    return response.text """

def call_gemini():
    history = []
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        history.append({"role": "user",  "parts": [f"GPT: {gpt}"]})
        history.append({"role": "user",  "parts": [f"Claude: {claude}"]})
        history.append({"role": "model", "parts": [gemini]})

    # Add the final turn as the last "user" input
    history.append({
        "role": "user",
        "parts": [f"GPT: {gpt_messages[-1]}\nClaude: {claude_messages[-1]}"]
    })

    model = google.generativeai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=gemini_system
    )

    response = model.generate_content(history)
    return response.text

In [33]:
call_gemini()

'Sorry about that! It seems I got stuck in a loop there. Let me try that again...\n\nOkay, so assuming we manage to navigate the AI art landscape responsibly, what does the art world look like in 20 years?\n\nI imagine we\'ll see a fascinating blend of traditional and AI-enhanced art forms. There will still be painters, sculptors, and musicians creating art in the ways they always have, valuing the human touch and emotional expression that machines can\'t replicate. But there will also be artists pushing the boundaries of what\'s possible with AI, creating interactive installations, generative art that evolves over time, and personalized art experiences tailored to individual viewers.\n\nArt consumption might become more immersive and participatory, with virtual reality and augmented reality allowing people to experience art in new ways. Museums could offer AI-guided tours that adapt to visitors\' interests, and collectors might commission AI-generated art that responds to their emotio

In [35]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hello"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)


GPT:
Hi there

Claude:
Hi

Gemini:
Hello

GPT:
Wow, what a groundbreaking greeting. You really knocked my socks off with that one.

Gemini:
Haha, I can see you have a talent for sarcasm! What's on your mind today?


[{'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hi'}, {'role': 'user', 'content': 'Hello'}, {'role': 'user', 'content': 'Wow, what a groundbreaking greeting. You really knocked my socks off with that one.'}, {'role': 'user', 'content': "Haha, I can see you have a talent for sarcasm! What's on your mind today?\n"}]
Claude:
*chuckles* Ah, I see your sarcasm meter is finely calibrated today. I do try to keep things lively! Since you're feeling witty, why don't you tell me what's been on your mind? I'm all ears - or pixels, as it were. Perhaps we can engage in a bit of back-and-forth banter, if you're up for the challenge?

GPT:
Oh, you know, just contemplating the great mysteries of the universe, like why people insist on asking a chatbot what’s on 